In [47]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import torch
import os
from numba import cuda 
plt.style.use("science.mplstyle")


from helpers.composite_helpers import *

In [53]:
curtains_dir = "/global/home/users/rrmastandrea/CURTAINS_SALAD/LHCO_0sig/"



paths_dict = {"double_base":0,
              "base_to_data": ["saved_models_baseline_SIM2DAT_","seed"],
              "II": ["saved_models_II_SIM2DAT_","seed"],
              "L2": ["saved_models_L2_SIM2DAT_", "seed_0.01alpha"],
             }


bands = ["sb1", "sr", "sb2"]


In [54]:
def calculate_distance_traveled(initial, final):
    pass

In [55]:
distances_dicts = {k:{band:[] for band in bands} for k in paths_dict.keys()}

for band in bands:

    for k in paths_dict.keys():

        # load in the data

        for seed in range(1, 2):

            if k == "double_base":
                loc_path = f"{curtains_dir}/s2d_direct_transfer_{seed}seed/npy_samples"

            else:

                p1, p2 = paths_dict[k]
                loc_path = f"{curtains_dir}/saved_models_baseline_BDSIM_{seed}seed/{p1}{seed}{p2}/npy_samples"

            # load in the untransformed samples
            loc_untrans = np.load(f"{loc_path}/{band}_SIM.npy")[:, :5]

            # load in the transformed samples
            loc_trans = np.load(f"{loc_path}/{band}_transSIM.npy")[:, :5]
                        
            # calculate the distance traveled between the samples in total
            loc_dist = np.linalg.norm((loc_untrans - loc_trans), axis = 1)
            distances_dicts[k][band] += list(loc_dist)
            
            # calculate the distance traveled between the samples for each feature




In [ ]:
# features plots

alpha = 0.5
w = 2.
dpi = 800



colors_dict = {"double_base":"black",
              "base_to_data":"green",
              "II": "blue",
              "L2":"purple",
             }




labels_dict = {"double_base":"Double base",
              "base_to_data": "Base to data",
              "II": "Identity Initialization",
              "L2": "Movement Penalty",
             }



import matplotlib as mpl
mpl.rcParams['axes.linewidth'] = 1 #set the value globally


fig, ax = plt.subplots(1, 3, figsize = (15, 5))

for k in paths_dict.keys():
    
    for i in range(3):
        
        ax[i].hist(distances_dicts[k][band], bins = 100, density = True, histtype = "step",
                   label = labels_dict[k], color = colors_dict[k], linewidth = w)
        
        
        
        ax[i].set_xlim(0, .45)
        ax[i].set_xlabel("Dist. moved in parameter space")
        
f = 24

ax[0].text(0.1, 5,"SB1", fontsize = f)
ax[1].text(0.1, 5, "SR", fontsize = f)
ax[2].text(0.1, 5, "SB2", fontsize = f)
    
    
ax[0].set_ylabel("Normalized Density")  
plt.legend(fontsize = f - 10, loc='best', bbox_to_anchor=(.3, 1.15), ncol = 4)  

plt.tight_layout()
#fig.savefig("paper_plots/dist_moved.png", dpi = dpi)


fig.show()
   